In [ ]:
import pandas as pd
import os

# Directory containing all the files
data_dir = 'D:/GUVI_Projects/My_Projects/new_horse/Horse'

# Function to load and check columns of horse and race data for a given year
def load_and_check_columns(year, data_dir):
    horse_file = os.path.join(data_dir, f'horses_{year}.csv')
    race_file = os.path.join(data_dir, f'races_{year}.csv')
    
    if os.path.exists(horse_file) and os.path.exists(race_file):
        horses = pd.read_csv(horse_file, low_memory=False)
        races = pd.read_csv(race_file, low_memory=False)
        
        print(f"Columns in horses_{year}.csv: {horses.columns}")
        print(f"Columns in races_{year}.csv: {races.columns}")
    else:
        print(f"Files for year {year} not found.")

# Check columns for each year from 1990 to 2020
for year in range(1990, 2021):
    load_and_check_columns(year, data_dir)


In [ ]:
import pandas as pd
import os

# Directory containing all the files
data_dir = 'D:/GUVI_Projects/My_Projects/new_horse/Horse'

# Function to load and merge horse and race data for a given year
def load_and_merge_yearly_data(year, data_dir):
    horse_file = os.path.join(data_dir, f'horses_{year}.csv')
    race_file = os.path.join(data_dir, f'races_{year}.csv')
    
    if os.path.exists(horse_file) and os.path.exists(race_file):
        horses = pd.read_csv(horse_file, low_memory=False)
        races = pd.read_csv(race_file, low_memory=False)
        
        # Ensure 'rid' exists in both datasets
        if 'rid' in horses.columns and 'rid' in races.columns:
            merged_data = pd.merge(horses, races, on='rid')
            print(f"Successfully merged data for year {year}")
            return merged_data
        else:
            print(f"'rid' column not found in one of the files for year {year}. Skipping...")
            return pd.DataFrame()
    else:
        print(f"Files for year {year} not found. Skipping...")
        return pd.DataFrame()  # Return an empty DataFrame if the files don't exist

# Load and merge data from 1990 to 2020
all_data = pd.DataFrame()

for year in range(1990, 2021):
    yearly_data = load_and_merge_yearly_data(year, data_dir)
    if not yearly_data.empty:
        all_data = pd.concat([all_data, yearly_data], ignore_index=True)

print("All yearly data loaded and merged.")
print("Columns in all_data:", all_data.columns)

# Load forward.csv
forward_file = os.path.join(data_dir, 'forward.csv')
forward_data = pd.read_csv(forward_file, low_memory=False)
print("Forward data loaded.")
print("Columns in forward_data:", forward_data.columns)

# Ensure necessary columns exist in both datasets before merging
merge_columns = ['course', 'title', 'rclass', 'horseName', 'trainerName', 'jockeyName']
if all(col in all_data.columns for col in merge_columns) and all(col in forward_data.columns for col in merge_columns):
    complete_data = pd.merge(all_data, forward_data, on=merge_columns, how='left')
    print("Forward data merged with all yearly data.")
else:
    print("One of the necessary columns not found in one of the datasets. Merging skipped.")
    complete_data = all_data

# Select important columns that are present in the DataFrame
important_columns = [
    'rid', 'horseName', 'trainerName', 'jockeyName', 'date', 'course', 'age', 'weight', 
    'distance', 'condition', 'position', 'decimalPrice', 'RPR', 'TR', 'OR'
]
existing_columns = [col for col in important_columns if col in complete_data.columns]
complete_data = complete_data.loc[:, existing_columns]
print("Selected important columns.")

# Handle missing values
complete_data.ffill(inplace=True)
print("Handled missing values using forward fill.")

# Convert categorical variables to numerical representations
categorical_columns = ['course', 'trainerName', 'jockeyName']
for column in categorical_columns:
    if column in complete_data.columns:
        complete_data[column] = complete_data[column].astype('category').cat.codes
print("Encoded categorical variables to numerical representations.")

# Save the final cleaned dataset
output_file = os.path.join(data_dir, 'cleaned_final_dataset.csv')
complete_data.to_csv(output_file, index=False)

print("Dataset creation completed and saved as 'cleaned_final_dataset.csv'.")


In [ ]:
import pandas as pd

data = pd.read_csv('D:/GUVI_Projects/My_Projects/new_horse/Horse/cleaned_final_dataset.csv')
print(data.describe())

In [ ]:
print(data.isnull().sum())
print(data.columns)

In [ ]:
import matplotlib.pyplot as plt

# Function to create and display histograms
def plot_histogram(data, column, title):
    plt.figure(figsize=(10, 6))
    plt.hist(data[column], bins=30, edgecolor='black', alpha=0.7)
    plt.title(title)
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

# List of columns to plot
columns_to_plot = ['weight', 'position', 'RPR', 'TR']

# Plot histograms for the specified columns
for column in columns_to_plot:
    plot_histogram(data, column, f'Distribution of {column.capitalize()}')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Selecting only numeric columns for the correlation matrix
numeric_data = data.select_dtypes(include=['float64', 'int64'])

# Creating the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Function to add value labels on bars
def add_value_labels(ax):
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.0f'),
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha = 'center', va = 'center',
                   xytext = (0, 10),
                   textcoords = 'offset points')

# Load the entire dataset
df = pd.read_csv('D:/GUVI_Projects/My_Projects/new_horse/Horse/cleaned_final_dataset.csv', low_memory=True)

# Ensure the columns are treated as strings
df['course'] = df['course'].astype(str)
df['trainerName'] = df['trainerName'].astype(str)
df['jockeyName'] = df['jockeyName'].astype(str)

# Count the occurrences of each unique value in the columns
course_counts = df['course'].value_counts().reset_index()
trainer_counts = df['trainerName'].value_counts().reset_index()
jockey_counts = df['jockeyName'].value_counts().reset_index()

# Rename the columns for clarity
course_counts.columns = ['course', 'count']
trainer_counts.columns = ['trainerName', 'count']
jockey_counts.columns = ['jockeyName', 'count']

# Example mapping dictionaries (replace with actual mappings if available)
trainer_name_mapping = {9907: "Trainer A", 15408: "Trainer B", 15396: "Trainer C"}
jockey_name_mapping = {4783: "Jockey A", 4740: "Jockey B", 15750: "Jockey C"}

# Replace IDs with names
trainer_counts['trainerName'] = trainer_counts['trainerName'].astype(int).map(trainer_name_mapping).fillna(trainer_counts['trainerName'])
jockey_counts['jockeyName'] = jockey_counts['jockeyName'].astype(int).map(jockey_name_mapping).fillna(jockey_counts['jockeyName'])

# Bar plot for Course
if not course_counts.empty:
    plt.figure(figsize=(12, 8))
    course_order = course_counts.sort_values('count', ascending=False).head(20)['course']
    ax = sns.barplot(data=course_counts.head(20), x='course', y='count', order=course_order)
    plt.xticks(rotation=90, fontsize=10)
    plt.xlabel('Course')
    plt.ylabel('Count')
    plt.title('Top 20 Courses by Count')
    add_value_labels(ax)
    plt.tight_layout()
    plt.show()
else:
    print("No data available for 'course'.")

# Bar plot for Trainer Names (top 20 for better visualization)
if not trainer_counts.empty:
    plt.figure(figsize=(12, 8))
    top_trainers = trainer_counts.nlargest(20, 'count')
    trainer_order = top_trainers.sort_values('count', ascending=False)['trainerName']
    ax = sns.barplot(data=top_trainers, y='trainerName', x='count', order=trainer_order)
    plt.xticks(rotation=90, fontsize=10)
    plt.xlabel('Count')
    plt.ylabel('Trainer Name')
    plt.title('Top 20 Trainers')
    add_value_labels(ax)
    plt.tight_layout()
    plt.show()
else:
    print("No data available for 'trainerName'.")

# Bar plot for Jockey Names (top 20 for better visualization)
if not jockey_counts.empty:
    plt.figure(figsize=(12, 8))
    top_jockeys = jockey_counts.nlargest(20, 'count')
    jockey_order = top_jockeys.sort_values('count', ascending=False)['jockeyName']
    ax = sns.barplot(data=top_jockeys, y='jockeyName', x='count', order=jockey_order)
    plt.xticks(rotation=90, fontsize=10)
    plt.xlabel('Count')
    plt.ylabel('Jockey Name')
    plt.title('Top 20 Jockeys')
    add_value_labels(ax)
    plt.tight_layout()
    plt.show()
else:
    print("No data available for 'jockeyName'.")

# Additional Visualizations

# Visualization for top 20 horse names by count
horse_counts = df['horseName'].value_counts().reset_index()
horse_counts.columns = ['horseName', 'count']

if not horse_counts.empty:
    plt.figure(figsize=(12, 8))
    top_horses = horse_counts.nlargest(20, 'count')
    horse_order = top_horses.sort_values('count', ascending=False)['horseName']
    ax = sns.barplot(data=top_horses, x='horseName', y='count', order=horse_order)
    plt.xticks(rotation=90, fontsize=10)
    plt.xlabel('Horse Name')
    plt.ylabel('Count')
    plt.title('Top 20 Horses by Count')
    add_value_labels(ax)
    plt.tight_layout()
    plt.show()
else:
    print("No data available for 'horseName'.")

# Visualization for race positions
position_counts = df['position'].value_counts().reset_index()
position_counts.columns = ['position', 'count']

if not position_counts.empty:
    plt.figure(figsize=(12, 8))
    position_order = position_counts.sort_values('count', ascending=False)['position']
    ax = sns.barplot(data=position_counts, x='position', y='count', order=position_order)
    plt.xticks(rotation=90, fontsize=10)
    plt.xlabel('Position')
    plt.ylabel('Count')
    plt.title('Race Positions by Count')
    add_value_labels(ax)
    plt.tight_layout()
    plt.show()
else:
    print("No data available for 'position'.")

# Interactive Plots using Plotly

# Interactive plot for top 20 courses by count
fig = px.bar(course_counts.head(20), x='course', y='count', title='Top 20 Courses by Count')
fig.show()

# Interactive plot for top 20 trainers by count
fig = px.bar(trainer_counts.head(20), x='trainerName', y='count', title='Top 20 Trainers by Count')
fig.show()

# Interactive plot for top 20 jockeys by count
fig = px.bar(jockey_counts.head(20), x='jockeyName', y='count', title='Top 20 Jockeys by Count')
fig.show()


In [ ]:
#Feature Engineering:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv('D:/GUVI_Projects/My_Projects/new_horse/Horse/cleaned_final_dataset.csv')

# Example feature engineering: Creating features based on past performances
df['win'] = (df['position'] == 1).astype(int)
df['place'] = df['position'].apply(lambda x: 1 if x <= 3 else 0)

# Aggregating performance metrics for horses
horse_performance = df.groupby('horseName').agg({
    'win': 'sum',
    'place': 'sum',
    'RPR': 'mean',
    'TR': 'mean',
    'weight': 'mean'
}).reset_index()

# Merging performance metrics back into the main dataset
df = df.merge(horse_performance, on='horseName', suffixes=('', '_avg'))

print("Feature engineering completed.")


In [ ]:
#Data Preprocessing for Modeling:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Selecting features and target variable
features = ['weight', 'RPR', 'TR', 'win_avg', 'place_avg', 'RPR_avg', 'TR_avg', 'weight_avg']
X = df[features]
y = df['win']  # Assuming the goal is to predict if the horse wins

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data preprocessing completed.")


In [ ]:
#Model Selection and Training:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Training a RandomForest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = rf_model.predict(X_test_scaled)

# Evaluation
print("Random Forest Model Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Random Forest Model Performance:
# Accuracy: 0.9022526430432796
#               precision    recall  f1-score   support

#            0       0.91      0.99      0.95   1113116
#            1       0.47      0.08      0.14    119081

#     accuracy                           0.90   1232197
#    macro avg       0.69      0.54      0.55   1232197
# weighted avg       0.87      0.90      0.87   1232197



In [ ]:
#Handling Imbalanced Data:
#Use techniques to handle imbalanced data, such as SMOTE.
from imblearn.over_sampling import SMOTE

# Applying SMOTE to the training data
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train_scaled, y_train)

# Retraining the model on balanced data
rf_model_res = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_res.fit(X_train_res, y_train_res)

# Predictions on original test data
y_pred_res = rf_model_res.predict(X_test_scaled)

# Evaluation
print("Random Forest Model with SMOTE Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred_res))
print(classification_report(y_test, y_pred_res))

# Confusion Matrix
conf_matrix_res = confusion_matrix(y_test, y_pred_res)
plt.figure(figsize=(10, 6))
sns.heatmap(conf_matrix_res, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix with SMOTE')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


# Random Forest Model with SMOTE Performance:
# Accuracy: 0.890455828085931
#               precision    recall  f1-score   support

#            0       0.92      0.96      0.94   1113116
#            1       0.38      0.22      0.28    119081

#     accuracy                           0.89   1232197
#    macro avg       0.65      0.59      0.61   1232197
# weighted avg       0.87      0.89      0.88   1232197




In [ ]:
#Hyperparameter Tuning:
#Optimize hyperparameters using Grid Search or Random Search.
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Hyperparameter tuning using Grid Search
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Reduce the number of parallel jobs to avoid memory issues
grid_search = GridSearchCV(estimator=rf_model_res, param_grid=param_grid, cv=3, n_jobs=1, verbose=2)
grid_search.fit(X_train_res, y_train_res)

print("Best parameters found: ", grid_search.best_params_)

# Retraining the model with best parameters
best_rf_model = grid_search.best_estimator_
y_pred_best = best_rf_model.predict(X_test_scaled)

# Evaluation
print("Random Forest Model with Best Parameters Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

# Confusion Matrix
conf_matrix_best = confusion_matrix(y_test, y_pred_best)
plt.figure(figsize=(10, 6))
sns.heatmap(conf_matrix_best, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix with Best Parameters')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Save the trained model and the final dataset for future use
import joblib

# Save the final dataset
final_dataset_path = 'D:/GUVI_Projects/My_Projects/new_horse/Horse/final_dataset.csv'
df.to_csv(final_dataset_path, index=False)
print("Final dataset saved.")

# Save the trained model
model_path = 'D:/GUVI_Projects/My_Projects/new_horse/Horse/best_rf_model.pkl'
joblib.dump(best_rf_model, model_path)
print("Trained model saved.")

# Fitting 3 folds for each of 16 candidates, totalling 48 fits
# [CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=15.0min